## **Basic** Background Subtraction using [OpenCV](https://www.opencv.org/)

![Screenshot](screenshot.png)

## Package inclusion for Python

In [1]:
import numpy as np
import cv2

## Open the webcam

We are going to use the [`cv2.VideoCapture` class](https://docs.opencv.org/master/d8/dfe/classcv_1_1VideoCapture.html) to grab frames from the webcam.

In [2]:
# Open the default camera (see the 0 below)
video_input = cv2.VideoCapture(0)

# Check VideoCapture
if not video_input.isOpened():
    raise IOError("OpenCV found no webcam, the program will terminate")

## Display the images from the camera

We create a window to display the images from the webcam

In [3]:
cv2.namedWindow("Webcam", cv2.WINDOW_GUI_EXPANDED) # Create a window
cv2.namedWindow("Background", cv2.WINDOW_GUI_EXPANDED) # Create a window
cv2.namedWindow("Absolute difference", cv2.WINDOW_GUI_EXPANDED) # Create a window
cv2.namedWindow("Foreground mask", cv2.WINDOW_GUI_EXPANDED) # Create a window
cv2.namedWindow("Foreground", cv2.WINDOW_GUI_EXPANDED) # Create a window

We display the images in a loop. We also need the background. We pick up the backbround by pressing the 'b' key on the keyboard.

We'll perform the background subtraction using an absolute difference between the background and the incoming frame. Then, we apply a threshold. It creates a binary mask of the foreground. We write the background subtraction as a function. But first, as we'll create a binary mask, I want to do some cleaning using mathematical morphology and use it via a new function.

In [4]:
def cleanBinaryImage(aBinaryImage: np.array, elementSize: int=5) -> np.array:

    element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,
                        (elementSize, elementSize))
    
    output = cv2.morphologyEx(aBinaryImage, cv2.MORPH_CLOSE, element)
    output = cv2.morphologyEx(aBinaryImage, cv2.MORPH_OPEN, element)

    return output;

In [5]:
def getForegroundMask(aBackground: np.array, aNewFrame: np.array, aThreshold: int=128) -> np.array:

    # Convert in greyscale
    grey_frame = cv2.cvtColor(aNewFrame, cv2.COLOR_BGR2GRAY)
    
    # Blur the image a bit
    grey_frame = cv2.medianBlur(grey_frame, 5)

    # Convert to float32
    grey_frame = grey_frame.astype(np.single)

    # Compute the foreground as the absolute difference
    foreground = aBackground - grey_frame;
    foreground = np.abs(foreground)
    
    # Normalise the foreground
    norm = np.zeros(foreground.shape)
    foreground = cv2.normalize(foreground, norm, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)

    # Display the foreground
    cv2.imshow("Absolute difference", foreground);

    # Apply a threshold to generate the foreground mask
    foreground_mask = cv2.threshold(foreground, aThreshold, 255, cv2.THRESH_BINARY)[1]

    # Use mathematical morphology to clean the binary image
    foreground_mask = cleanBinaryImage(foreground_mask, 15)

    # Display the foreground mask
    cv2.imshow("Foreground mask", foreground_mask)

    # Return the mask
    return foreground_mask.astype(np.uint8)

As we'll use a threshold, I'll add a trackbar to control it.

In [6]:
def callback(value: int):
    global threshold_value
    threshold_value = value

In [7]:
threshold_value = 128;
cv2.createTrackbar("Threshold: ", "Foreground mask", threshold_value, 255, callback)

[ WARN:0] global /home/conda/feedstock_root/build_artifacts/libopencv_1633800893877/work/modules/highgui/src/window.cpp (703) createTrackbar UI/Trackbar(Threshold: @Foreground mask): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.


In [8]:
key = -1
background = None

while key != 27 and key != 'q':

    # Grab a new frame
    ret, frame = video_input.read()

    # Make sure everything went well
    if frame is None:
        video_input.release() # We are now done with the camera, stop it
        raise IOError("OpenCV cannot grab a new frame from the camera, the program will terminate")
       
    # Display the image
    cv2.imshow("Webcam", frame)

    # This is the background
    if key == 'b' or key == 98:

        background = frame
        background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
        cv2.imshow("Background", background)
        background = cv2.medianBlur(background, 5)
        background = background.astype(np.single)

    # This is not the background
    else:

        # The background exists
        if background is not None:

            # Update the threshold
            foreground_mask = getForegroundMask(background, frame, threshold_value)

            # Apply the foreground mask
            foreground_mask = cv2.cvtColor(foreground_mask, cv2.COLOR_GRAY2BGR)
            clean = cv2.bitwise_and(foreground_mask, frame)
            cv2.imshow("Foreground", clean);

    key = cv2.waitKey(1)

We don't need the window, destroy it

In [9]:
cv2.destroyAllWindows() # Destroy all the created windows

We are now done with the camera, stop it

In [10]:
video_input.release()